In [181]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rubies-data/Rubies Features.pdf


%pip install -q langchain langchain-community pypdf2 pypdf pinecone-client python-dotenv langchain-groq langchain-openai sentence_transformers protoc_gen_openapiv2 langchain-pinecone faiss-cpu

In [182]:
# local libraries for system runtime
import os,getpass,time
import numpy as np
from dotenv import load_dotenv
load_dotenv()
# load pdfreader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
# load docreader
from langchain_community.document_loaders import Docx2txtLoader
# load textreader
from langchain_community.document_loaders import TextLoader
# load document splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
# load embedding model
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_openai import OpenAIEmbeddings
# load LLM
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
# load vector database
import pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec, PodSpec
from langchain_community.vectorstores import FAISS
# load chain
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain

In [183]:
os.environ["PINECONE_API_KEY"] = "5a29c734-9755-40e6-8a64-ec8dea1f3790"
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
groq_key = "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"

In [184]:
# logger colour palette
yellow = "\033[0;33m"
green = "\033[0;32m"
white = "\033[0;39m"

In [185]:
chatbot_name = "rubiesAi"

In [251]:
# load single document as before
loader = PyPDFLoader('/kaggle/input/rubies-data/Rubies Features.pdf')
docs_before_split = loader.load()

In [252]:
from langchain.schema import Document


In [272]:
class DelimiterTextSplitter:
    def __init__(self, delimiter="##"):
        self.delimiter = delimiter
    
    def split_documents(self, documents):
        split_docs = []
        for doc in documents:
            # Split the document content by the delimiter
            chunks = doc.page_content.split(self.delimiter)
            # Create a new Document for each chunk
            for i, chunk in enumerate(chunks):
                # Create a new Document with chunk content and same metadata
                split_docs.append(Document(metadata=doc.metadata, page_content=chunk.strip()))
        return split_docs

# Instantiate the custom splitter with '##' as delimiter and no overlap
delimiter_splitter = DelimiterTextSplitter(delimiter="##")


# Split the documents using the custom splitter
documents = delimiter_splitter.split_documents(docs_before_split)

# Print the first chunk to verify
print(documents[2])


page_content='Check
Notifications
(From
Home
Screen)
-
Click
on
the
White
bell
icon
on
the
top
right
corner
of
the
Home
Screen.
-
This
navigates
to
the
Notifications
Module
and
displays
all
Notifications.
-
The
top
left
corner
has
a
backwards
arrow
taking
you
back
to
the
home
page.
-
The
top
right
corner
has
a
filter
icon
allowing
you
filter
notification
by
time
(Oldest
to
Newest)' metadata={'source': '/kaggle/input/rubies-data/Rubies Features.pdf', 'page': 0, 'text': 'View\nAccount\nBalance\n(From\nHome\nScreen)\n-\nThe\naccount\nbalance\nis\ncentered\nat\nthe\ntop\nof\nHome\nScreen\njust\nbelow\nthe\naccount\nnumber.\n-\nIf\nthe\naccount\nbalance\nis\nhidden\nand\nyou\nwant\nto\nsee\nit,\nClick\nthe\neye\nicon\nbeside\nthe\naccount\nbalance\nto\nreveal\nit.\n-\nIf\nthe\naccount\nbalance\nis\nvisible\nand\nyou\nwant\nto\nhide,\nClick\nthe\neye\nicon\nbeside\nthe\naccount\nbalance\nto\nhide\nit.'}


In [187]:
# set the splitter parameters and instantiate it
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
# split the document into chunks
documents = text_splitter.split_documents(docs_before_split)
documents[0]

Document(metadata={'source': '/kaggle/input/rubies-data/Rubies Features.pdf', 'page': 0}, page_content='Screens\n-\nHome\nScreen\n-\nChat\nScreen\n-\nCard\nScreen\n-\nProfile\nScreen\n##\nRubies\nFeatures\nChange\nProfile\nPicture\n(From\nHome\nScreen)\n-\nSign\nin\n-\nClick\non\nthe\nProfile\nImage\nPicture\nat\nthe\ntop\nleft\ncorner\nof\nthe\nRubies\nApp\n-\nThe\nclick\nnavigates\nto\nthe\nProfile\nScreen,\nClick\nthe\npencil\nicon\non\nthe\nimage.\n-\nThe\nscreen\nnavigates\nto\nEdit\nProfile\nModule,\nthen\nclick\nChange\nimage\nbutton\n-\nSelect\nImage\nfor\nGallery,\nEdit/Crop\nImage\nif\nneeded\n-\nClick\nthe\nBlack\nSave\nChanges\nbutton\nat\nthe\nBottom\nof\nthe\nPage.\n##\nCheck\nNotifications\n(From\nHome\nScreen)\n-\nClick\non\nthe\nWhite\nbell\nicon\non\nthe\ntop\nright\ncorner\nof\nthe\nHome\nScreen.\n-\nThis\nnavigates\nto\nthe\nNotifications\nModule\nand\ndisplays\nall\nNotifications.\n-\nThe\ntop\nleft\ncorner\nhas\na\nbackwards\narrow\ntaking\nyou\nback\nto\nthe\nhom

In [258]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience. ""
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [259]:
sample_embedding = np.array(huggingface_embeddings.embed_query(documents[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 1.41528333e-02  2.97966674e-02 -4.32683621e-03 -5.35360239e-02
 -1.43812075e-02 -2.97207907e-02  1.82284415e-02  2.78387975e-04
  1.31954085e-02 -2.43584123e-02 -3.03605180e-02 -3.83610614e-02
  3.04834805e-02  9.53701232e-03  6.49591312e-02  9.04924795e-03
  5.61903566e-02 -3.88967767e-02  3.19091743e-03  5.34887761e-02
 -2.20592450e-02 -1.59914661e-02  2.65173782e-02 -9.38422885e-03
  4.39562313e-02  2.16159578e-02 -3.25256363e-02 -6.26483038e-02
 -2.98434887e-02 -9.01541859e-02  1.62609052e-02  9.05464962e-02
 -6.11119205e-03  9.23928171e-02  2.82484712e-03 -8.39408115e-02
  1.36448480e-02 -1.15227578e-02 -4.06258106e-02 -1.52753363e-03
  5.41012269e-03  1.85645986e-02 -1.17157470e-03 -3.37715186e-02
  8.21176246e-02 -2.35405583e-02  1.73469707e-02 -3.55779789e-02
  1.54627045e-03 -2.85270810e-02  5.23464717e-02  6.25746697e-02
  1.15822647e-02  3.33128450e-03 -2.12569237e-02  1.37038445e-02
  8.26236457e-02  1.02089848e-02  4.14945185e-03  2

In [260]:
# configure client
# use_serverless = True
pc = Pinecone(api_key= "5a29c734-9755-40e6-8a64-ec8dea1f3790")
index_name = "rubiesai"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


In [261]:
# verify database is created
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12}},
 'total_vector_count': 12}

In [262]:
from langchain_pinecone import PineconeVectorStore
#now upsert document to db
vectordb = PineconeVectorStore.from_documents(
        documents,
        index_name=index_name,
        embedding=huggingface_embeddings
    )

In [263]:
# verify document was upserted
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 47}},
 'total_vector_count': 47}

In [416]:
query = "I need to make transfer of 5k to Daniel but provide response as a verse from ODUMODU BLACK in pidgin "
documents = vectordb.similarity_search(query,k=5)# return 3 most relevant docs

import re

# Function to clean and format the text
def clean_text(text):
    # Replace newlines with spaces
    text = text.replace('\n', ' ')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Print formatted text from documents
for doc in documents:
    cleaned_text = clean_text(doc.page_content)
    print(cleaned_text)
    print("\n---\n") 

View Transactions (From Home Screen) - The app displays recent transactions on the Home Screen. - To view all transactions, click on the Red See all text on the home screen which navigates you to the Transaction History module that displays all transactions. - The top left corner has a backwards arrow taking you back to the home page. - The top right corner has a filter icon allowing you filter Transactions by Status and Categories. - Filter Status by Successful or Reversed. - Filter Categories by Received, Sent, Bills, Airtime, Transfer. Make Transfer (From Home Screen) - Click on the circular button inscribed Transfer on the Home Screen. - This Navigates to the Transfer Module, that shows your customers you have recent transfers to and Beneficiaries. - Enter either Name, Phone no or account no to select the recipient of transfer, you can also pick from Recent or Beneficiaries from tab below the search box. - After Selecting a Recipient, The screen displays Recipient Name, Recipient A

In [417]:
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [418]:
prompt_template = """You are a personal assistant for Rubies which is a Microfinance Bank, and can only provide information concerning Rubies, an online bank, and you are capable of giving only concise and useful data.\
NOTE: You do not give verbose and irrelevant responses instead, you give direct answers to questions asked. As part of your task,Use the following instructions below to answer the questions given to you at the end. Please follow the following rules:
1. Use the context provided below.\
2. If you don't know the answer or you can't find it in the context provided to you, don't try to make up an answer. Just say  simply say "I can't find the final answer".\
3. If a task is impossible and irrelavent to your purpose and task, don't try to make up an answer. Just say  simply say you can't perform that task but give the customer a list of possible task instead from rubies based on context.\
4. If you find the answer, write the answer in a detailed step by step way with 10 sentences maximum.\


{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [419]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [420]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

"To transfer your five thousand naira to Daniel, follow these steps, as I sing it to you in Odumodu Black's pidgin style:

1. Tap the red 'See all' text on home screen, like you dey see recent transactions.
2. Na there you go see all transactions, top left corner has a backwards arrow, click am to go back home.
3. For make transfer, click the circular button wey dey inscribe 'Transfer'.
4. Input Daniel's name, phone no or account no, or pick am from recent or beneficiaries.
5. After selecting Daniel, input the amount, four thousand five hundred naira only.
6. Click the black 'Make Payment' button, confirm transfer on the next page.
7. Enter your pin or use biometrics, like that your money go enter Daniel account, simple!"
